# 3. 텐서 구조체

## 3.2 텐서: 다차원 배열

### 3.2.1 파이썬 리스트에서 파이토치 텐서로

- 텐서 자료구조를 사용해 이미지와 시계열 데이터, 문장 나타내는 것이 더 효율적

In [1]:
a = [1.0, 2.0, 1.0]

In [2]:
a[0]

1.0

In [3]:
a[2] = 3.0
a

[1.0, 2.0, 3.0]

### 3.2.2 첫 텐서 만들어보기

In [4]:
import torch 
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [5]:
a[1]

tensor(1.)

In [6]:
float(a[1])

1.0

In [7]:
a[2] = 2.0
a

tensor([1., 1., 2.])

### 3.2.1 텐서의 핵심

- 파이토치 텐서나 넘파이 배열은 언박싱(unboxing)된 배열
- C언어의 숫자 타입 포함한 연속적인 메모리 할당 및 뷰 제공
- 각 요소는 32비트(4바이트) float 타입

In [8]:
points = torch.zeros(6) 
points[0] = 4.0 
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0

In [9]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

In [10]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [11]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [12]:
points.shape

torch.Size([3, 2])

In [13]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [14]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [15]:
points[0, 1]

tensor(1.)

In [16]:
points[0]

tensor([4., 1.])

## 3.3 텐서 인덱싱

In [17]:
some_list = list(range(6))
some_list[:]
some_list[1:4]
some_list[1:]
some_list[:4]
some_list[:-1]
some_list[1:4:2]

[1, 3]

In [18]:
points[1:]
points[1:, :]
points[1:, 0]
points[None] 
# 길이가 1인 차원을 추가함. unsqueeze와 동일

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

## 3.4 이름이 있는 텐서


In [19]:
# 이미지 데이터를 흑백으로 변환한다고 가정, 여러 색상별 가중치(weights)를 보고 하나의 밝기 값을 뽑아내는 과정이 됨

img_t = torch.randn(3, 5, 5) 
# torch.randn()은 평균이 0이고 표준편차가 1인 가우시간 정규분포를 이용해 생성
# 각각이 [채널 크기, 행 크기, 열 크기]가 됨

weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [20]:
# 흑백 이미지로부터 RGB값을 담을 세 번째 채널 차원을 더한 코드 (배치 크기 2로 가정)
batch_t = torch.randn(2, 3, 5, 5) 

In [21]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape
# RGB 채널은 두 코드에서 모두 세 번째 차원에 있음

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

- 브로드캐스팅(broadcasting) : 서로 크기가 다른 행렬들이 사칙연산 수행할 수 있도록 크기 맞춰줌
- unsqueeze(dim = 0) : 차원이 1인 차원을 생성하는 함수, 어느 차원에 생성할 것인지 지정해주어야 함

In [22]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
# unsqueeze_(-1) 은 바꿔치기(in-place)연산으로,
# the shape of the tensor was modified directly without going through the PyTorch dispatcher 라고 한다.

img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

- einsum

In [23]:
# ...(점 세 개)로 변수명 없이 합을 구하는 브로드캐스팅 방법
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

#### 텐서 이름 지정

In [92]:
# tensor나 rand 같은 텐서 팩토리 함수에 이름 지정 가능
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [25]:
# 텐서를 먼저 만들고 나중에 이름을 지정
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


- align_as : 빠진 차원을 채우고 존재하는 차원을 올바른 순서로 바꿔줌

In [26]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [27]:
# sum처럼 차원 인수를 허용하는 함수들은 이름 붙은 차원도 받아들임
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [28]:
# 이름이 다른 차원을 결합하려면 오류 발생
gray_named = (img_named[..., :3] * weights_named).sum('channels')

RuntimeError: Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.

In [ ]:
# 이름 있는 텐서를 사용하는 연산을 함수 밖에서도 사용하려면, 차원 이름에 None 넣어 이름 없는 텐서 만듦
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

## 3.5 텐서 요소 타입
- 박싱(boxing) : 파이썬에서 참조 카운터까지 만들어 부동소수점 수를 완전한 파이썬 객체로 변환하는 것
- 박싱은 수백만 개 넘어가는 큰 숫자에서 상당히 비효율적

### 3.5.1 dtype으로 숫자 타입 지정하기
- tensor나 zeros, ones 같은 텐서 생성자 실행시 넘겨주는 dtype 인자로 데이터 타입 정할 수 있음
- 기본 데이터 타입은 32비트 부동소수점

### 3.5.3 텐서의 dtype 속성 관리

In [29]:
# dtype 인자 정확하게 전달해야 함 
double_points = torch.ones(10,2, dtype = torch.double)
short_points = torch.tensor([[1,2], [3,4]], dtype = torch.short)

In [30]:
short_points.dtype

torch.int16

In [31]:
# 캐스팅 메소드 사용
double_points = torch.zeros(10,2).double()
short_points = torch.ones(10,2).short()

In [32]:
# to 메소드 사용
# 타입 외에도 추가적인 인자 지정 가능 (3.9절 참고)
double_points = torch.zeros(10,2).to(torch.double)
short_points = torch.ones(10,2).to(dtype = torch.short)

In [33]:
# 여러 타입 연산 섞이면 제일 큰 타입으로 만들어짐
points_64 = torch.rand(5, dtype=torch.double) 
# rand는 텐서 요소를 0과 1사이 임의의 수로 초기화
points_short = points_64.to(torch.short)
points_64 * points_short

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

## 3.6 텐서 API

In [34]:
# torch 모듈로 호출한 transpose 함수
a = torch.ones(3,2)
a_t = torch.transpose(a, 0, 1)


a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [35]:
# 텐서 메소드 이용
a = torch.ones(3,2)
a_t = a.transpose(0,1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

- 텐서 연산<br>
<img src="https://user-images.githubusercontent.com/107118671/188539128-d4201cb1-885e-4b81-a721-bf3a1ba8ce6e.png" width="50%" height="50%" align="left" />

## 3.7 텐서를 저장소 관점에서 머릿속에 그려보기
### 3.7.1 저장 공간 인덱싱
- 텐서 내부 값은 실제로는 torch.Storage 인스턴스로 관리, 연속적인 메모리 조각으로 할당된 상태
- 파이토치의 Tensor 객체는 Storage 객체에 대한 뷰 역할 담당

In [36]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage._TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [37]:
# 차원에 무관하게 실제 저장 공간 레이아웃은 1차원
points_storage = points.storage()
points_storage[0]

4.0

In [38]:
points.storage()[1]

1.0

In [39]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_storage = points.storage()
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

### 3.7.2 저장된 값을 수정하기: 텐서 내부 연산

In [40]:
a = torch.ones(3,2)

In [41]:
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

## 3.8 텐서 메타데이터: 사이즈, 오프셋, 스트라이드
- 사이즈 : 넘파이 용어로는 shape, 텐서의 각 차원 별로 들어가는 요소의 수를 표시한 튜플
- 오프셋 : 텐서의 첫 번째 요소를 가리키는 색인 값과 동일
- 스트라이드 : 각 차원에서 다음 요소 가리키고 싶을 때 실제 저장 공간에서 몇 개의 요소 건너뛰는지 알려주는 숫자<br>
<img src="https://user-images.githubusercontent.com/107118671/188542247-4de63122-a0ea-4b4d-be40-78c80885f77b.png" width="70%" height="70%" align="left" />


### 3.8.1 다른 텐서의 저장 공간에 대한 뷰 만들기

In [42]:
points = torch.tensor(([4.0, 1.0], [5.0, 3.0], [2.0, 1.0]))
second_point = points[1]
second_point.storage_offset()

2

In [43]:
# 텐서 객체의 shape 속성값과 동일
second_point.size()

torch.Size([2])

In [44]:
second_point.shape

torch.Size([2])

In [45]:
points.stride()
# 다음 행 가려면 2칸 뛰어야 하고, 다음 열 가려면 1칸 뛰면 됨

(2, 1)

- 2차원 텐서에서 요소 i,j에 접근한다면 저장 공간상으로는 (storage_offset + stride[0] * i + stride[1] * j)번째 요소 의미

In [46]:
# 특정 부분만 정의한 서브텐서(subtensor)
second_point = points[1]
second_point.size()

torch.Size([2])

In [47]:
second_point.storage_offset()

2

In [48]:
second_point.stride()

(1,)

In [49]:
points = torch.tensor(([4.0, 1.0], [5.0, 3.0], [2.0, 1.0]))
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [50]:
# 서브텐서를 새 텐서로 복제
points = torch.tensor(([4.0, 1.0], [5.0, 3.0], [2.0, 1.0]))
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

### 3.8.2 복사 없이 전치하기

In [51]:
points = torch.tensor(([4.0, 1.0], [5.0, 3.0], [2.0, 1.0]))
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [52]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [53]:
# 같은 공간 가지는지 확인
id(points.storage()) == id(points_t.storage())

True

In [54]:
# 두 텐서는 차원 정보와 스트라이드만 다름
points.stride()

(2, 1)

In [55]:
points_t.stride()

(1, 2)

- 전치 연산 : 새로운 메모리는 할당되지 않고 원래 것과 다른 스트라이드 순서를 가진 새로운 Tensor 인스턴스 만듦

### 3.8.3 더 높은 차원에서의 전치 연산

In [56]:
some_t = torch.ones(3,4,5)
transpose_t = some_t.transpose(0,2)
some_t.shape

torch.Size([3, 4, 5])

In [57]:
transpose_t.shape

torch.Size([5, 4, 3])

In [58]:
some_t.stride()

(20, 5, 1)

In [59]:
transpose_t.stride()

(1, 5, 20)

### 3.8.4 인접한 텐서
- contiguous : 가장 오른쪽 차원에서 시작해 증가되는 형태로 저장소에 값이 펼쳐진 텐서 (2차원 텐서에서 열을 따라 이동)
- 데이터 지역성(data locality)관점에서 CPU 메모리 접근 효율 좋음

In [60]:
points.is_contiguous()

True

In [61]:
points_t.is_contiguous()

False

In [62]:
points = torch.tensor(([4.0, 1.0], [5.0, 3.0], [2.0, 1.0]))
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [63]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage._TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [64]:
points_t.stride()

(1, 2)

In [65]:
# contiguous 메소드 통해 인접한 텐서로 만들 수 있음 (텐서 내용 동일하나 값의 배치와 스트라이드가 바뀜)
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [66]:
points_t_cont.stride()

(3, 1)

In [67]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.storage._TypedStorage(dtype=torch.float32, device=cpu) of size 6]

## 3.9 텐서를 GPU로 옮기기
###3.9.1 텐서 디바이스 속성 관리

In [68]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
points_gpu = points.to(device='cuda')

In [ ]:
points_gpu = points.to(device='cuda:0')

In [ ]:
points = 2 * points
points_gpu = 2 * points.to(device='cuda')

In [69]:
points_gpu = points_gpu + 4

NameError: name 'points_gpu' is not defined

In [70]:
points_cpu = points_gpu.to(device='cpu')

NameError: name 'points_gpu' is not defined

In [71]:
points_gpu = points.cuda()
points_gpu = points.cuda(0)
points_cpu = points_gpu.cpu()

# to 메소드를 사용할 때 device와 dtype 인자를 동시에 사용해 데이터 타입과 위치를 동시에 변경 가능

AssertionError: Torch not compiled with CUDA enabled

## 3.10 넘파이 호환

In [72]:
points = torch.ones(3,4)
points_np = points.numpy()
points_np
# 데이터가 cpu ram 영역에 있는 한, numpy 메소드를 아무 추가 비용 없이 사용 가능
# 넘파이 배열 수정 시, 원래 텐서에도 적용
# 텐서가 gpu에 있다면 cpu 영역으로 복사해 배열 만듦

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [73]:
points = torch.from_numpy(points_np)

# 파이토치 기본 숫자 타입은 32비트 부동소수점, 넘파이는 64비트

## 3.11 일반화된 텐서도 텐서다

- 분배(dispatching) 메커니즘에 의해, 파이토치는 텐서가 cpu, gpu 어디에 있어도 알맞은 연산 함수 호출 가능

## 3.12 텐서 직렬화

- 파일 포맷 자체가 파이토치로만 읽기 가능

In [74]:
torch.save(points, '../data/p1ch3/ourpoints.t')

In [75]:
with open('../data/p1ch3/ourpoints.t','wb') as f:
    torch.save(points, f)

In [76]:
points = torch.load('../data/p1ch3/ourpoints.t')

In [77]:
with open('../data/p1ch3/ourpoints.t', 'rb') as f:
    points = torch.load(f)

### 3.12.1 h5py로 HDF5 병렬화하기
- conda install h5py

In [78]:
import h5py

In [79]:
f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy())
f.close()

In [80]:
f = h5py.File('../data/p1ch3/ourpoints.hdf5', 'r')
dset = f['coords']
last_points = dset[-2:]

In [81]:
last_points = torch.from_numpy(dset[-2:])
f.close()

## 3.13 결론
- 지금까지 부동소수점 수 표현을 위한 모든 것을 살펴보았음
- 다음 4장에서는 파이토치로 실제 데이터를 표현하는 법 배울 예정

## 3.14 연습 문제

In [82]:
a = torch.tensor(list(range(9)))
print(f'사이즈는 {a.size()}') 
print(f'오프셋은 {a.storage_offset()}')
print(f'스트라이드는 {a.stride()}') 

사이즈는 torch.Size([9])
오프셋은 0
스트라이드는 (1,)


In [83]:
b = a.view(3,3)
b
# view 메소드의 역할은 똑같은 데이타를 가지고 다른 shape의 결과를 만드는 것 (단, 원래 텐서가 contiguous해야 사용 가능) 

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [84]:
id(a.storage()) == id(b.storage())

True

In [85]:
c = b[1:, 1:]
print(f'사이즈는 {c.size()}') 
print(f'오프셋은 {c.storage_offset()}')
print(f'스트라이드는 {c.stride()}') 

사이즈는 torch.Size([2, 2])
오프셋은 4
스트라이드는 (3, 1)


In [86]:
import math
math.sqrt(a)

ValueError: only one element tensors can be converted to Python scalars

In [87]:
torch.sqrt(a)

tensor([0.0000, 1.0000, 1.4142, 1.7321, 2.0000, 2.2361, 2.4495, 2.6458, 2.8284])

In [88]:
a

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [89]:
a_f = a.type(torch.float)
a_f

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8.])

In [90]:
torch.sqrt_(a_f)

tensor([0.0000, 1.0000, 1.4142, 1.7321, 2.0000, 2.2361, 2.4495, 2.6458, 2.8284])

In [91]:
a, a_f

(tensor([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 tensor([0.0000, 1.0000, 1.4142, 1.7321, 2.0000, 2.2361, 2.4495, 2.6458, 2.8284]))